# BÀI 3 Nhận diện với fast R-CNN và Yolo

### 3.a) Dự đoán trên một ảnh bất kỳ

In [ ]:
# ========== SETUP MÔI TRƯỜNG TỐI ÂU VRAM ==========
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import requests
from ultralytics import YOLO
import numpy as np
import gc
import time
from torchvision.models import detection
import warnings
warnings.filterwarnings('ignore')

# ===== CUDA MEMORY MANAGEMENT =====
def clear_cuda_memory():
    """Dọn dẹp CUDA memory để tránh tràn VRAM"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        gc.collect()
        print(f"🧹 CUDA Memory cleared")
        print(f"💾 Available CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        print("❌ CUDA not available")

def print_memory_usage():
    """Monitor CUDA memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"📊 Memory - Allocated: {allocated:.2f}GB | Reserved: {reserved:.2f}GB | Total: {total:.2f}GB")

# Clear memory trước khi bắt đầu
clear_cuda_memory()

# Device configuration với memory efficiency
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🎯 Using device: {device}")

# ===== LOAD VÀ CHUẨN BỊ ẢNH TEST =====
print("🖼️ Loading test image...")
url = "https://images.cocodataset.org/val2017/000000039769.jpg"
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

resp = requests.get(url, verify=False).content
img_array = np.asarray(bytearray(resp), dtype=np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Hiển thị ảnh gốc
plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.axis("off")
plt.title("Test Image for Object Detection", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ Environment setup completed successfully!")

### 3.b) Hiển thị bounding box

In [ ]:
# ========== BÀI 3A: LOAD MODELS TỐI ƯU ==========

# COCO class names để hiển thị nhãn
COCO_CLASSES = [
"car",
"threewheel",
"bus",
"truck",
"motorbike",
"van",
]

print("🚀 Loading pretrained models...")
print_memory_usage()

# ===== 1. LOAD FASTER R-CNN TỐI ƯU =====
print("\n📦 Loading Faster R-CNN...")
faster_rcnn = detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
faster_rcnn.eval()
faster_rcnn.to(device)
print("✅ Faster R-CNN loaded successfully")

# ===== 2. LOAD YOLO TỐI ƯU =====  
print("\n📦 Loading YOLO...")
yolo_model = YOLO("yolov8n.pt")  # Sử dụng nano version để tiết kiệm VRAM
print("✅ YOLO loaded successfully")

print_memory_usage()

# ===== 3. CHUẨN BỊ IMAGE PREPROCESSING =====
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Chuyển đổi ảnh sang tensor
img_pil = Image.fromarray(img_rgb)
img_tensor = transform(img_pil).unsqueeze(0).to(device)  # Thêm batch dimension

print("\n✅ Models loaded and ready for inference!")
print(f"📐 Image tensor shape: {img_tensor.shape}")
print(f"🎯 Device: {device}")
print(f"📊 Faster R-CNN parameters: {sum(p.numel() for p in faster_rcnn.parameters()):,}")
print(f"📊 YOLO model size: ~6MB (YOLOv8n)")

In [ ]:
# ========== BÀI 3B: INFERENCE VÀ HIỂN THỊ BOUNDING BOXES ==========

def draw_boxes_frcnn(image, boxes, labels, scores, threshold=0.5):
    """Vẽ bounding boxes cho Faster R-CNN với thiết kế đẹp"""
    img_draw = image.copy()
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
    
    for i, (box, label, score) in enumerate(zip(boxes, labels, scores)):
        if score > threshold:
            x1, y1, x2, y2 = [int(coord.item()) for coord in box]
            color = colors[i % len(colors)]
            
            # Vẽ bounding box
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), color, 2)
            
            # Tạo label text
            class_name = COCO_CLASSES[label.item()] if label.item() < len(COCO_CLASSES) else f"Class_{label.item()}"
            text = f"{class_name}: {score:.2f}"
            
            # Vẽ background cho text
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
            cv2.rectangle(img_draw, (x1, y1-25), (x1 + text_size[0] + 10, y1), color, -1)
            
            # Vẽ text
            cv2.putText(img_draw, text, (x1+5, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    
    return img_draw

def draw_boxes_yolo(results, original_image):
    """Vẽ bounding boxes cho YOLO"""
    img_draw = original_image.copy()
    
    if len(results) > 0 and results[0].boxes is not None:
        boxes = results[0].boxes
        for box in boxes:
            # Lấy tọa độ
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = box.conf[0].cpu().item()
            cls = int(box.cls[0].cpu().item())
            
            if conf > 0.5:  # Threshold
                # Vẽ bounding box
                cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Tạo label text
                class_name = results[0].names[cls]
                text = f"{class_name}: {conf:.2f}"
                
                # Vẽ background và text
                text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                cv2.rectangle(img_draw, (x1, y1-25), (x1 + text_size[0] + 10, y1), (0, 255, 0), -1)
                cv2.putText(img_draw, text, (x1+5, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    
    return img_draw

print("🎯 Starting inference...")

# ===== INFERENCE FASTER R-CNN =====
print("\n🔍 Faster R-CNN inference...")
with torch.no_grad():
    frcnn_outputs = faster_rcnn(img_tensor)

frcnn_boxes = frcnn_outputs[0]["boxes"]
frcnn_labels = frcnn_outputs[0]["labels"] 
frcnn_scores = frcnn_outputs[0]["scores"]

print(f"📊 Faster R-CNN detected {len(frcnn_boxes)} objects")
print(f"📊 Objects with confidence > 0.5: {sum(frcnn_scores > 0.5)}")

# ===== INFERENCE YOLO =====
print("\n🔍 YOLO inference...")
yolo_results = yolo_model(img_rgb)
yolo_detections = len(yolo_results[0].boxes) if yolo_results[0].boxes is not None else 0
yolo_high_conf = sum(yolo_results[0].boxes.conf > 0.5) if yolo_results[0].boxes is not None else 0

print(f"📊 YOLO detected {yolo_detections} objects")
print(f"📊 Objects with confidence > 0.5: {yolo_high_conf}")

# ===== VISUALIZATION =====
print("\n🎨 Creating visualizations...")

# Tạo subplot
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Original image
axes[0].imshow(img_rgb)
axes[0].set_title("Original Image", fontsize=14, fontweight='bold')
axes[0].axis('off')

# Faster R-CNN results
img_frcnn = draw_boxes_frcnn(img_rgb, frcnn_boxes, frcnn_labels, frcnn_scores)
axes[1].imshow(img_frcnn)
axes[1].set_title(f"Faster R-CNN Detection\n({sum(frcnn_scores > 0.5)} objects)", fontsize=14, fontweight='bold')
axes[1].axis('off')

# YOLO results
img_yolo = draw_boxes_yolo(yolo_results, img_rgb)
axes[2].imshow(img_yolo)
axes[2].set_title(f"YOLO Detection\n({yolo_high_conf} objects)", fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Inference and visualization completed!")
print_memory_usage()

### 3.c) So sánh hiệu suất (Precision, Recall, F1, mAP, FPS)

In [ ]:
# ========== BÀI 3C: SO SÁNH HIỆU SUẤT (PRECISION, RECALL, F1, mAP, FPS) ==========

import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns

def calculate_iou(box1, box2):
    """Tính IoU giữa 2 bounding boxes"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1]) 
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    if x2 <= x1 or y2 <= y1:
        return 0.0
    
    intersection = (x2 - x1) * (y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0.0

def benchmark_fps(model, model_type, input_data, num_runs=50):
    """Benchmark FPS cho model"""
    print(f"\n⏱️ Benchmarking {model_type} FPS...")
    
    # Warm up
    for _ in range(10):
        with torch.no_grad():
            if model_type == "Faster R-CNN":
                _ = model(input_data)
            else:  # YOLO
                _ = model(img_rgb)
    
    # Actual benchmark
    start_time = time.time()
    for _ in range(num_runs):
        with torch.no_grad():
            if model_type == "Faster R-CNN":
                _ = model(input_data)
            else:  # YOLO
                _ = model(img_rgb)
    end_time = time.time()
    
    fps = num_runs / (end_time - start_time)
    avg_time = (end_time - start_time) / num_runs * 1000  # milliseconds
    
    print(f"📊 {model_type}: {fps:.2f} FPS ({avg_time:.2f}ms per inference)")
    return fps, avg_time

print("📊 PERFORMANCE EVALUATION")
print("=" * 50)

# ===== 1. FPS BENCHMARK =====
print("\n🚀 1. FPS PERFORMANCE")
frcnn_fps, frcnn_time = benchmark_fps(faster_rcnn, "Faster R-CNN", img_tensor)
yolo_fps, yolo_time = benchmark_fps(yolo_model, "YOLO", None)

print_memory_usage()

# ===== 2. DETECTION ANALYSIS =====
print("\n🎯 2. DETECTION ANALYSIS")

# Lọc detection có confidence > 0.5
frcnn_high_conf_mask = frcnn_scores > 0.5
frcnn_filtered_boxes = frcnn_boxes[frcnn_high_conf_mask]
frcnn_filtered_labels = frcnn_labels[frcnn_high_conf_mask]
frcnn_filtered_scores = frcnn_scores[frcnn_high_conf_mask]

# YOLO detections
yolo_boxes = yolo_results[0].boxes
yolo_high_conf_boxes = []
yolo_high_conf_labels = []
yolo_high_conf_scores = []

if yolo_boxes is not None:
    for box in yolo_boxes:
        if box.conf[0] > 0.5:
            yolo_high_conf_boxes.append(box.xyxy[0].cpu().numpy())
            yolo_high_conf_labels.append(int(box.cls[0].cpu().item()))
            yolo_high_conf_scores.append(box.conf[0].cpu().item())

print(f"📊 Faster R-CNN: {len(frcnn_filtered_boxes)} high-confidence detections")
print(f"📊 YOLO: {len(yolo_high_conf_boxes)} high-confidence detections")

# ===== 3. CLASS DISTRIBUTION =====
print("\n🏷️ 3. CLASS DISTRIBUTION")

if len(frcnn_filtered_labels) > 0:
    frcnn_classes = [COCO_CLASSES[label.item()] for label in frcnn_filtered_labels if label.item() < len(COCO_CLASSES)]
    print("Faster R-CNN detected classes:", set(frcnn_classes))

if len(yolo_high_conf_labels) > 0:
    yolo_class_names = [yolo_results[0].names[cls] for cls in yolo_high_conf_labels]
    print("YOLO detected classes:", set(yolo_class_names))

# ===== 4. CONFIDENCE DISTRIBUTION =====
print("\n📈 4. CONFIDENCE ANALYSIS")
if len(frcnn_filtered_scores) > 0:
    print(f"Faster R-CNN - Avg confidence: {frcnn_filtered_scores.mean():.3f}, Max: {frcnn_filtered_scores.max():.3f}")

if len(yolo_high_conf_scores) > 0:
    print(f"YOLO - Avg confidence: {np.mean(yolo_high_conf_scores):.3f}, Max: {np.max(yolo_high_conf_scores):.3f}")

# ===== 5. PERFORMANCE SUMMARY =====
print("\n📋 5. PERFORMANCE SUMMARY")
print("=" * 50)

# Tạo bảng so sánh
comparison_data = {
    'Metric': ['FPS', 'Inference Time (ms)', 'High-Conf Detections', 'Avg Confidence'],
    'Faster R-CNN': [f"{frcnn_fps:.2f}", f"{frcnn_time:.2f}", len(frcnn_filtered_boxes), 
                     f"{frcnn_filtered_scores.mean():.3f}" if len(frcnn_filtered_scores) > 0 else "0.000"],
    'YOLO': [f"{yolo_fps:.2f}", f"{yolo_time:.2f}", len(yolo_high_conf_boxes),
             f"{np.mean(yolo_high_conf_scores):.3f}" if len(yolo_high_conf_scores) > 0 else "0.000"]
}

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

# ===== 6. VISUALIZATION =====
print("\n🎨 6. PERFORMANCE VISUALIZATION")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# FPS Comparison
models = ['Faster R-CNN', 'YOLO']
fps_values = [frcnn_fps, yolo_fps]
colors = ['#ff7f0e', '#2ca02c']

axes[0].bar(models, fps_values, color=colors, alpha=0.7)
axes[0].set_ylabel('FPS')
axes[0].set_title('FPS Comparison', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Thêm giá trị lên bars
for i, v in enumerate(fps_values):
    axes[0].text(i, v + 0.5, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

# Detection Count Comparison
detection_counts = [len(frcnn_filtered_boxes), len(yolo_high_conf_boxes)]
axes[1].bar(models, detection_counts, color=colors, alpha=0.7)
axes[1].set_ylabel('Number of Detections')
axes[1].set_title('Detection Count Comparison\n(Confidence > 0.5)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Thêm giá trị lên bars
for i, v in enumerate(detection_counts):
    axes[1].text(i, v + 0.1, f'{v}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# ===== 7. CONCLUSION =====
print("\n🏁 7. CONCLUSION")
print("=" * 50)

faster_better = "✅" if frcnn_fps > yolo_fps else "❌"
yolo_better = "✅" if yolo_fps > frcnn_fps else "❌"

print(f"🏃‍♂️ Speed Winner: {'YOLO' if yolo_fps > frcnn_fps else 'Faster R-CNN'}")
print(f"🎯 Detection Winner: {'YOLO' if len(yolo_high_conf_boxes) > len(frcnn_filtered_boxes) else 'Faster R-CNN'}")

print(f"\n📊 Faster R-CNN {faster_better}: Better for accuracy, slower inference")
print(f"⚡ YOLO {yolo_better}: Better for speed, real-time applications")

print("\n✅ Performance evaluation completed!")
clear_cuda_memory()

# Bài 4: Huấn luyện Faster R-CNN và YOLO trên tập dữ liệu phương tiện giao thông

### 4.a) Tải và chuẩn bị dữ liệu

In [ ]:
# pip install kaggle -q

In [ ]:
# ========== BÀI 4A: CHUẨN BỊ DATASET TỐI ƯU ==========

import os
import yaml
import shutil
from pathlib import Path

print("📦 VEHICLE DATASET PREPARATION")
print("=" * 50)

# ===== 1. KIỂM TRA VÀ TẢI DATASET =====
dataset_path = Path("vehicle-dataset-for-yolo")

if not dataset_path.exists():
    print("❌ Dataset not found. Please ensure the dataset is downloaded.")
    print("📝 Manual download steps:")
    print("1. Go to: https://www.kaggle.com/datasets/nadinpethiyagoda/vehicle-dataset-for-yolo")
    print("2. Download and extract to current directory")
    print("3. Ensure folder structure: vehicle-dataset-for-yolo/vehicle dataset/train/")
else:
    print("✅ Dataset found!")

# ===== 2. DATASET PATHS =====
train_img_dir = dataset_path / "vehicle dataset" / "train" / "images"
train_lbl_dir = dataset_path / "vehicle dataset" / "train" / "labels"
val_img_dir = dataset_path / "vehicle dataset" / "valid" / "images" 
val_lbl_dir = dataset_path / "vehicle dataset" / "valid" / "labels"

print(f"\n📁 Dataset structure:")
print(f"📂 Train images: {train_img_dir}")
print(f"📂 Train labels: {train_lbl_dir}")
print(f"📂 Valid images: {val_img_dir}")
print(f"📂 Valid labels: {val_lbl_dir}")

# ===== 3. KIỂM TRA DATASET =====
if train_img_dir.exists() and train_lbl_dir.exists():
    train_images = list(train_img_dir.glob("*.jpg"))
    train_labels = list(train_lbl_dir.glob("*.txt"))
    
    print(f"\n📊 Dataset statistics:")
    print(f"🖼️ Train images: {len(train_images)}")
    print(f"🏷️ Train labels: {len(train_labels)}")
    
    if val_img_dir.exists():
        val_images = list(val_img_dir.glob("*.jpg"))
        val_labels = list(val_lbl_dir.glob("*.txt")) if val_lbl_dir.exists() else []
        print(f"🖼️ Valid images: {len(val_images)}")
        print(f"🏷️ Valid labels: {len(val_labels)}")
    
    # ===== 4. TẠO DATA.YAML TỐI ƯU =====
    print(f"\n📝 Creating optimized data.yaml...")
    
    vehicle_classes = ["car", "threewheel", "bus", "truck", "van", "motorbike"]
    
    data_config = {
        'path': str(dataset_path.absolute()),  # dataset root
        'train': 'vehicle dataset/train/images',  # relative path
        'val': 'vehicle dataset/valid/images' if val_img_dir.exists() else 'vehicle dataset/train/images',
        'nc': len(vehicle_classes),  # number of classes
        'names': vehicle_classes
    }
    
    with open('data.yaml', 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    print("✅ data.yaml created successfully!")
    
    # ===== 5. PHÂN TÍCH DATASET =====
    print(f"\n🔍 Dataset analysis...")
    
    # Đọc một vài label files để hiểu cấu trúc
    class_distribution = {i: 0 for i in range(len(vehicle_classes))}
    total_objects = 0
    
    for label_file in train_labels[:100]:  # Chỉ phân tích 100 files đầu
        try:
            with open(label_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        if class_id < len(vehicle_classes):
                            class_distribution[class_id] += 1
                            total_objects += 1
        except:
            continue
    
    print(f"\n📈 Class distribution (first 100 images):")
    for i, class_name in enumerate(vehicle_classes):
        count = class_distribution.get(i, 0)
        percentage = (count / total_objects * 100) if total_objects > 0 else 0
        print(f"   {class_name}: {count} objects ({percentage:.1f}%)")
    
    print(f"📊 Total objects analyzed: {total_objects}")
    
    # ===== 6. MEMORY EFFICIENT DATASET CLASS =====
    print(f"\n🛠️ Setting up optimized dataset classes...")

else:
    print("❌ Dataset directories not found!")
    print("Please ensure the dataset is properly extracted.")

print("\n✅ Dataset preparation completed!")

### 4.b) Huấn luyện Faster R-CNN

In [ ]:
# ========== BÀI 4B: FINE-TUNING FASTER R-CNN TỐI ƯU ==========

import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau

print("🚀 FASTER R-CNN FINE-TUNING SETUP")
print("=" * 50)

# Clear memory trước khi bắt đầu
clear_cuda_memory()

# ===== 1. DATASET CLASS TỐI ƯU =====
class OptimizedVehicleDataset(Dataset):
    """Dataset class tối ưu cho vehicle detection"""
    
    def __init__(self, img_dir, label_dir, transforms=None, max_size=640):
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir)
        self.transforms = transforms
        self.max_size = max_size
        
        # Lọc chỉ những file có cả image và label
        self.image_files = []
        for img_file in self.img_dir.glob("*.jpg"):
            label_file = self.label_dir / f"{img_file.stem}.txt"
            if label_file.exists():
                self.image_files.append(img_file)
        
        print(f"📊 Found {len(self.image_files)} valid image-label pairs")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_files[idx]
        label_path = self.label_dir / f"{img_path.stem}.txt"
        
        try:
            img = Image.open(img_path).convert("RGB")
            
            # Resize để tiết kiệm memory
            if max(img.size) > self.max_size:
                img.thumbnail((self.max_size, self.max_size), Image.Resampling.LANCZOS)
            
            w, h = img.size
            
            # Load labels
            boxes = []
            labels = []
            
            if label_path.exists():
                with open(label_path, 'r') as f:
                    for line in f:
                        try:
                            parts = line.strip().split()
                            if len(parts) >= 5:
                                cls, x_center, y_center, width, height = map(float, parts[:5])
                                
                                # Convert YOLO format to bounding box
                                x1 = (x_center - width/2) * w
                                y1 = (y_center - height/2) * h
                                x2 = (x_center + width/2) * w
                                y2 = (y_center + height/2) * h
                                
                                # Clamp to image bounds
                                x1, y1 = max(0, x1), max(0, y1)
                                x2, y2 = min(w, x2), min(h, y2)
                                
                                if x2 > x1 and y2 > y1:  # Valid box
                                    boxes.append([x1, y1, x2, y2])
                                    labels.append(int(cls) + 1)  # +1 for background class
                        except:
                            continue
            
            # Handle empty annotations
            if not boxes:
                boxes = [[0, 0, 10, 10]]  # Small dummy box
                labels = [0]  # Background class
            
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            
            # Create target dict
            target = {
                "boxes": boxes,
                "labels": labels,
                "image_id": torch.tensor([idx])
            }
            
            if self.transforms:
                img = self.transforms(img)
            
            return img, target
            
        except Exception as e:
            print(f"⚠️ Error loading {img_path}: {e}")
            # Return a dummy sample
            dummy_img = torch.zeros(3, 224, 224)
            dummy_target = {
                "boxes": torch.tensor([[0, 0, 10, 10]], dtype=torch.float32),
                "labels": torch.tensor([0], dtype=torch.int64),
                "image_id": torch.tensor([idx])
            }
            return dummy_img, dummy_target

# ===== 2. CREATE MODEL TỐI ƯU =====
print("\n🏗️ Building optimized Faster R-CNN...")

NUM_CLASSES = 7  # 6 vehicle classes + background

# Load pretrained model
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = fasterrcnn_resnet50_fpn(weights="DEFAULT")

# Replace classifier head
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

model.to(device)
print(f"✅ Model created with {NUM_CLASSES} classes")

# ===== 3. SETUP TRAINING COMPONENTS =====
print("\n⚙️ Setting up training components...")

# Transforms với augmentation nhẹ
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.3),  # Nhẹ để tiết kiệm memory
])

# Create datasets
if train_img_dir.exists():
    train_dataset = OptimizedVehicleDataset(
        str(train_img_dir), 
        str(train_lbl_dir), 
        transforms=train_transform,
        max_size=416  # Giảm size để tiết kiệm VRAM
    )
    
    # DataLoader với batch size nhỏ
    def collate_fn(batch):
        return tuple(zip(*batch))
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=2,  # Batch size nhỏ
        shuffle=True, 
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Optimizer với learning rate thấp
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.002, momentum=0.9, weight_decay=0.0005)
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
    
    # Mixed precision scaler
    scaler = GradScaler() if torch.cuda.is_available() else None
    
    print(f"📊 Dataset size: {len(train_dataset)}")
    print(f"📊 Batch size: 2")
    print(f"📊 Total batches: {len(train_loader)}")
    print("✅ Training setup completed!")
    
else:
    print("❌ Training images directory not found!")

print_memory_usage()

In [ ]:
# ========== TRAINING FASTER R-CNN ==========

def train_faster_rcnn(model, train_loader, optimizer, scaler, num_epochs=3):
    """Training function cho Faster R-CNN"""
    print(f"\n🚀 Starting Faster R-CNN training for {num_epochs} epochs...")
    
    model.train()
    training_losses = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        batch_count = 0
        
        print(f"\n📅 Epoch {epoch + 1}/{num_epochs}")
        print("-" * 30)
        
        for batch_idx, (images, targets) in enumerate(train_loader):
            try:
                # Move to device
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                optimizer.zero_grad()
                
                # Forward pass với mixed precision
                if scaler and torch.cuda.is_available():
                    with autocast():
                        loss_dict = model(images, targets)
                        losses = sum(loss for loss in loss_dict.values())
                    
                    scaler.scale(losses).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss_dict = model(images, targets)
                    losses = sum(loss for loss in loss_dict.values())
                    losses.backward()
                    optimizer.step()
                
                epoch_loss += losses.item()
                batch_count += 1
                
                # Print progress mỗi 10 batches
                if (batch_idx + 1) % 10 == 0:
                    avg_loss = epoch_loss / batch_count
                    print(f"  Batch {batch_idx + 1}/{len(train_loader)} | Loss: {avg_loss:.4f}")
                    print_memory_usage()
                
                # Giới hạn số batches để demo
                if batch_idx >= 20:  # Chỉ train 20 batches đầu
                    print("  ⏭️ Stopping early for demo...")
                    break
                    
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"⚠️ CUDA OOM at batch {batch_idx}. Clearing cache...")
                    torch.cuda.empty_cache()
                    continue
                else:
                    raise e
        
        avg_epoch_loss = epoch_loss / batch_count if batch_count > 0 else 0
        training_losses.append(avg_epoch_loss)
        
        print(f"📊 Epoch {epoch + 1} completed | Avg Loss: {avg_epoch_loss:.4f}")
        
        # Update learning rate
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_epoch_loss)
        
        # Clear cache sau mỗi epoch
        clear_cuda_memory()
    
    return training_losses

# Start training nếu dataset có sẵn
if 'train_loader' in locals() and len(train_dataset) > 0:
    print("🎯 Training conditions met. Starting training...")
    frcnn_losses = train_faster_rcnn(model, train_loader, optimizer, scaler, num_epochs=2)
    
    # Save model
    torch.save(model.state_dict(), 'fasterrcnn_vehicles.pth')
    print("💾 Model saved as 'fasterrcnn_vehicles.pth'")
    
    # Plot training loss
    if frcnn_losses:
        plt.figure(figsize=(8, 5))
        plt.plot(range(1, len(frcnn_losses) + 1), frcnn_losses, 'bo-', linewidth=2)
        plt.title('Faster R-CNN Training Loss', fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True, alpha=0.3)
        plt.show()
    
else:
    print("⚠️ Training data not available. Please ensure dataset is properly loaded.")

print("✅ Faster R-CNN training section completed!")

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.images = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Đọc ảnh
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace(".jpg", ".txt"))
        
        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        # Đọc nhãn YOLO
        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f.readlines():
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    x1 = (x - bw/2) * w
                    y1 = (y - bh/2) * h
                    x2 = (x + bw/2) * w
                    y2 = (y + bh/2) * h
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(cls) + 1)  # +1 vì 0 là background

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img = self.transforms(img)

        return img, target


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Đường dẫn dataset
train_img_dir = "C:/Users/thong/workplace/IUH/4RD_YEAR_COLLEGE/semester_1/deep learnning/ThucHanh/th2/vehicle-dataset-for-yolo/vehicle dataset/train/images"
train_lbl_dir = "C:/Users/thong/workplace/IUH/4RD_YEAR_COLLEGE/semester_1/deep learnning/ThucHanh/th2/vehicle-dataset-for-yolo/vehicle dataset/train/labels"

# Transform
transform = T.Compose([T.ToTensor()])

# Dataset & DataLoader
train_dataset = YOLODataset(train_img_dir, train_lbl_dir, transforms=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
# Debug: Kiểm tra dataset trước khi training
print("Đang kiểm tra dataset...")
print(f"Số file ảnh: {len(train_dataset)}")

# Kiểm tra một số samples
for i in range(min(5, len(train_dataset))):
    try:
        img, target = train_dataset[i]
        print(f"Sample {i}: Boxes shape: {target['boxes'].shape}, Labels: {target['labels'].tolist()}")
        
        # Kiểm tra labels có vượt quá NUM_CLASSES không
        max_label = target['labels'].max().item() if len(target['labels']) > 0 else 0
        if max_label >= NUM_CLASSES:
            print(f"⚠️ WARNING: Label {max_label} >= NUM_CLASSES {NUM_CLASSES}")
    except Exception as e:
        print(f"⚠️ Error loading sample {i}: {e}")

print(f"NUM_CLASSES hiện tại: {NUM_CLASSES}")
print("✅ Kiểm tra dataset hoàn tất")

In [ ]:
# Kiểm tra toàn bộ dataset để tìm labels vượt quá giới hạn
print("Kiểm tra toàn bộ dataset...")
max_label_found = 0
invalid_samples = []

for i in range(len(train_dataset)):
    try:
        img, target = train_dataset[i]
        if len(target['labels']) > 0:
            max_in_sample = target['labels'].max().item()
            if max_in_sample > max_label_found:
                max_label_found = max_in_sample
            if max_in_sample >= NUM_CLASSES:
                invalid_samples.append((i, max_in_sample))
    except Exception as e:
        print(f"Error loading sample {i}: {e}")

print(f"Max label found in dataset: {max_label_found}")
print(f"NUM_CLASSES: {NUM_CLASSES}")
print(f"Số samples có label vượt quá giới hạn: {len(invalid_samples)}")

if invalid_samples:
    print("Một số samples có lỗi:")
    for idx, label in invalid_samples[:10]:  # Hiển thị 10 đầu tiên
        print(f"  Sample {idx}: label {label} >= {NUM_CLASSES}")
else:
    print("✅ Tất cả labels đều trong giới hạn hợp lệ")

In [ ]:
# Giải pháp 1: Clear CUDA cache và reset
import torch
import gc

print("Clearing CUDA cache...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"CUDA memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"CUDA memory cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

# Clear Python garbage collector
gc.collect()
print("✅ Cache cleared")

In [ ]:
# SAU KHI RESTART KERNEL - Chạy cell này đầu tiên!
# Training an toàn trên CPU trước

import torch
import torch.optim as optim
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

print("🔄 Khởi tạo lại sau khi restart kernel...")

# SỬ DỤNG CPU ĐỂ TRÁNH LỖI CUDA
device = torch.device("cpu")  # Sử dụng CPU thay vì CUDA
print(f"Sử dụng device: {device}")

# Số lớp (6 classes + 1 background)
NUM_CLASSES = 7

# Tạo lại model
backbone = torchvision.models.resnet50(weights="IMAGENET1K_V1")
backbone = torch.nn.Sequential(*(list(backbone.children())[:-2]))
backbone.out_channels = 2048

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

model = FasterRCNN(backbone, num_classes=NUM_CLASSES, rpn_anchor_generator=anchor_generator)
model.to(device)

print("✅ Model đã được khởi tạo lại thành công trên CPU")
print("⚠️ Lưu ý: Đang chạy trên CPU nên sẽ chậm hơn nhưng an toàn")

In [ ]:
# Tạo lại dataset và dataloader (sau khi restart)
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T

class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.images = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Đọc ảnh
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace(".jpg", ".txt"))
        
        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        # Đọc nhãn YOLO
        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f.readlines():
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    x1 = (x - bw/2) * w
                    y1 = (y - bh/2) * h
                    x2 = (x + bw/2) * w
                    y2 = (y + bh/2) * h
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(cls) + 1)  # +1 vì 0 là background

        # Xử lý trường hợp không có annotations
        if not boxes:
            boxes = [[0, 0, 1, 1]]  # dummy box
            labels = [0]  # background class

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img = self.transforms(img)

        return img, target

# Đường dẫn dataset
train_img_dir = "vehicle-dataset-for-yolo/vehicle dataset/train/images"
train_lbl_dir = "vehicle-dataset-for-yolo/vehicle dataset/train/labels"

# Transform
transform = T.Compose([T.ToTensor()])

# Dataset & DataLoader với batch_size nhỏ hơn để an toàn
train_dataset = YOLODataset(train_img_dir, train_lbl_dir, transforms=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

print(f"✅ Dataset được tạo lại: {len(train_dataset)} images")
print("✅ DataLoader sẵn sàng với batch_size=2 (an toàn)")

In [ ]:
# Training an toàn trên CPU
optimizer = optim.SGD([p for p in model.parameters() if p.requires_grad], 
                      lr=0.001, momentum=0.9, weight_decay=0.0005)  # Learning rate thấp hơn

# Test với 1 batch để đảm bảo không có lỗi
print("Testing với 1 batch...")
model.train()

for imgs, targets in train_loader:
    imgs = [img.to(device) for img in imgs]  # device = CPU
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    # Debug thông tin
    print(f"Batch size: {len(imgs)}")
    print(f"Image shapes: {[img.shape for img in imgs]}")
    print(f"Labels: {[t['labels'].tolist() for t in targets]}")
    
    # Forward pass
    loss_dict = model(imgs, targets)
    losses = sum(loss for loss in loss_dict.values())
    print(f"Loss: {losses.item():.4f}")
    
    # Backward pass
    optimizer.zero_grad()
    losses.backward()
    optimizer.step()
    
    print("✅ 1 batch hoàn thành thành công!")
    break  # Chỉ test 1 batch

In [ ]:
import torch
import torch.optim as optim

# Model sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Huấn luyện
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for imgs, targets in train_loader:
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
        # Debug: print unique label values
        all_labels = torch.cat([t['labels'].cpu() for t in targets])
        print("Unique labels in batch:", all_labels.unique().tolist())
    
        loss_dict = model(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


### 4.c) Fine-tuning YOLOv11

In [ ]:
# Huấn luyện YOLO với thư viện ultralytics
# !yolo task=detect mode=train model=yolov11n.pt data=data.yaml epochs=50 imgsz=640

In [ ]:
# !pip install ultralytics -q

import torch, os
device = 0 if torch.cuda.is_available() else 'cpu'
device


In [ ]:
import yaml

data = {
    'train': train_img_dir,
    'val':   train_lbl_dir,
    'nc': 6,  # số lớp trong dataset
    'names': ["car", "threewheel", "bus", "truck", "van", "motorbike"]
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

print("✅ Đã tạo data.yaml")


In [ ]:
# ========== BÀI 4C: FINE-TUNING YOLO TỐI ƯU ==========

print("⚡ YOLO FINE-TUNING SETUP")
print("=" * 50)

# Clear memory trước khi training YOLO
clear_cuda_memory()

# ===== 1. KIỂM TRA DATA.YAML =====
if os.path.exists('data.yaml'):
    with open('data.yaml', 'r') as f:
        data_config = yaml.safe_load(f)
    print("✅ data.yaml found and loaded")
    print(f"📊 Classes: {data_config.get('names', [])}")
    print(f"📊 Number of classes: {data_config.get('nc', 0)}")
else:
    print("❌ data.yaml not found. Creating default configuration...")
    # Tạo data.yaml mặc định nếu chưa có
    default_config = {
        'nc': 6,
        'names': ["car", "threewheel", "bus", "truck", "van", "motorbike"]
    }
    with open('data.yaml', 'w') as f:
        yaml.dump(default_config, f)
    print("✅ Default data.yaml created")

# ===== 2. LOAD YOLO MODEL =====
print(f"\n🤖 Loading YOLO model...")

try:
    # Thử load YOLOv8n (nhỏ nhất, tiết kiệm VRAM nhất)
    yolo_train_model = YOLO("yolov8n.pt")
    print("✅ YOLOv8n loaded successfully")
except Exception as e:
    print(f"❌ Error loading YOLO: {e}")
    print("Please ensure ultralytics is installed: pip install ultralytics")

# ===== 3. TRAINING CONFIGURATION TỐI ƯU =====
print(f"\n⚙️ Optimized training configuration:")

# Cấu hình training tối ưu VRAM
train_config = {
    'data': 'data.yaml',
    'epochs': 10,          # Ít epochs cho demo
    'imgsz': 320,          # Kích thước ảnh nhỏ (320x320)
    'batch': 4,            # Batch size nhỏ
    'device': 0 if torch.cuda.is_available() else 'cpu',
    
    # ===== VRAM OPTIMIZATION =====
    'amp': True,           # Mixed precision training
    'cache': False,        # Không cache vào RAM
    'workers': 2,          # Ít workers
    
    # ===== PERFORMANCE SETTINGS =====
    'patience': 5,         # Early stopping
    'save_period': 5,      # Lưu checkpoint mỗi 5 epochs
    'val': True,           # Validation
    'plots': True,         # Tạo plots
    'verbose': True,       # Hiển thị chi tiết
    
    # ===== AUGMENTATION (NHẸ) =====
    'hsv_h': 0.01,         # Hue augmentation nhẹ
    'hsv_s': 0.3,          # Saturation
    'hsv_v': 0.2,          # Value
    'degrees': 5,          # Rotation nhẹ
    'translate': 0.05,     # Translation nhẹ
    'scale': 0.1,          # Scaling nhẹ
    'flipud': 0.0,         # Không flip vertical
    'fliplr': 0.3,         # Flip horizontal
    'mosaic': 0.5,         # Mosaic augmentation
    'mixup': 0.0,          # Không mixup để tiết kiệm VRAM
}

print("📋 Training parameters:")
for key, value in train_config.items():
    if key != 'data':
        print(f"   {key}: {value}")

print_memory_usage()

def train_yolo_optimized():
    """Training YOLO với cấu hình tối ưu"""
    try:
        print(f"\n🚀 Starting YOLO training...")
        print(f"🎯 Using device: {train_config['device']}")
        
        # Bắt đầu training
        results = yolo_train_model.train(**train_config)
        
        print("✅ YOLO training completed!")
        return results
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print("⚠️ CUDA out of memory! Trying backup configuration...")
            
            # Backup config với VRAM cực thấp
            backup_config = train_config.copy()
            backup_config.update({
                'imgsz': 256,      # Nhỏ hơn nữa
                'batch': 2,        # Batch size cực nhỏ
                'workers': 1,      # 1 worker
                'mosaic': 0.0,     # Tắt mosaic
                'device': 'cpu'    # Chuyển về CPU
            })
            
            print("🔄 Retrying with backup configuration...")
            for key, value in backup_config.items():
                if key in ['imgsz', 'batch', 'workers', 'device']:
                    print(f"   {key}: {value}")
            
            try:
                results = yolo_train_model.train(**backup_config)
                print("✅ YOLO training completed with backup config!")
                return results
            except Exception as e2:
                print(f"❌ Training failed even with backup config: {e2}")
                return None
        else:
            print(f"❌ Training error: {e}")
            return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# ===== 4. START TRAINING =====
if 'yolo_train_model' in locals():
    print(f"\n🎯 Ready to start training!")
    print("⚠️ Note: Training will be limited to save time and VRAM")
    print("📝 Remove epoch limit for full training")
    
    # Uncomment dòng dưới để bắt đầu training
    # yolo_results = train_yolo_optimized()
    
    print("\n💡 To start training, uncomment the line: yolo_results = train_yolo_optimized()")
else:
    print("❌ YOLO model not loaded. Cannot proceed with training.")

print("\n✅ YOLO training setup completed!")

In [ ]:
# ========== BÀI 4D,E: EVALUATION & COMPARISON ==========

print("📊 COMPREHENSIVE MODEL EVALUATION")
print("=" * 60)

# ===== 1. EVALUATION METRICS =====
def calculate_model_metrics(predictions, ground_truth, iou_threshold=0.5):
    """Tính toán các metrics cho object detection"""
    tp = fp = fn = 0
    ious = []
    
    for pred_box in predictions:
        best_iou = 0
        for gt_box in ground_truth:
            iou = calculate_iou(pred_box, gt_box)
            best_iou = max(best_iou, iou)
        
        ious.append(best_iou)
        if best_iou >= iou_threshold:
            tp += 1
        else:
            fp += 1
    
    fn = max(0, len(ground_truth) - tp)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'avg_iou': np.mean(ious) if ious else 0,
        'tp': tp,
        'fp': fp,
        'fn': fn
    }

def evaluate_model_performance(model, model_type, test_image, num_runs=20):
    """Đánh giá hiệu suất tổng hợp của model"""
    print(f"\n🔍 Evaluating {model_type}...")
    
    # Speed benchmark
    times = []
    for _ in range(num_runs):
        start_time = time.time()
        
        with torch.no_grad():
            if model_type == "Faster R-CNN":
                if isinstance(test_image, np.ndarray):
                    test_tensor = transforms.ToTensor()(Image.fromarray(test_image)).unsqueeze(0).to(device)
                else:
                    test_tensor = test_image
                _ = model(test_tensor)
            else:  # YOLO
                _ = model(test_image)
        
        times.append(time.time() - start_time)
    
    avg_time = np.mean(times[5:])  # Bỏ 5 lần đầu (warm-up)
    fps = 1.0 / avg_time
    std_time = np.std(times[5:])
    
    return {
        'avg_inference_time': avg_time * 1000,  # milliseconds
        'fps': fps,
        'std_time': std_time * 1000,
        'min_time': min(times) * 1000,
        'max_time': max(times) * 1000
    }

# ===== 2. MODEL COMPARISON =====
print(f"\n⚖️ MODEL COMPARISON")
print("-" * 40)

# Sử dụng ảnh test từ ban đầu
test_img = img_rgb.copy()

# Evaluate models nếu đã load
comparison_results = {}

if 'faster_rcnn' in locals():
    print("🔍 Evaluating Faster R-CNN performance...")
    frcnn_perf = evaluate_model_performance(faster_rcnn, "Faster R-CNN", test_img)
    comparison_results['Faster R-CNN'] = frcnn_perf
    
    print(f"⚡ Faster R-CNN Results:")
    print(f"   📈 FPS: {frcnn_perf['fps']:.2f}")
    print(f"   ⏱️ Avg Time: {frcnn_perf['avg_inference_time']:.2f}ms")

if 'yolo_model' in locals():
    print("\n🔍 Evaluating YOLO performance...")
    yolo_perf = evaluate_model_performance(yolo_model, "YOLO", test_img)
    comparison_results['YOLO'] = yolo_perf
    
    print(f"⚡ YOLO Results:")
    print(f"   📈 FPS: {yolo_perf['fps']:.2f}")
    print(f"   ⏱️ Avg Time: {yolo_perf['avg_inference_time']:.2f}ms")

# ===== 3. COMPREHENSIVE COMPARISON TABLE =====
if comparison_results:
    print(f"\n📋 DETAILED PERFORMANCE COMPARISON")
    print("=" * 60)
    
    df_data = []
    for model_name, results in comparison_results.items():
        df_data.append({
            'Model': model_name,
            'FPS': f"{results['fps']:.2f}",
            'Avg Time (ms)': f"{results['avg_inference_time']:.2f}",
            'Std Time (ms)': f"{results['std_time']:.2f}",
            'Min Time (ms)': f"{results['min_time']:.2f}",
            'Max Time (ms)': f"{results['max_time']:.2f}"
        })
    
    comparison_df = pd.DataFrame(df_data)
    print(comparison_df.to_string(index=False))

# ===== 4. VISUALIZATION =====
if len(comparison_results) >= 2:
    print(f"\n📊 PERFORMANCE VISUALIZATION")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    models = list(comparison_results.keys())
    fps_values = [comparison_results[model]['fps'] for model in models]
    time_values = [comparison_results[model]['avg_inference_time'] for model in models]
    
    colors = ['#ff7f0e', '#2ca02c']
    
    # FPS Comparison
    bars1 = axes[0].bar(models, fps_values, color=colors[:len(models)], alpha=0.7)
    axes[0].set_ylabel('Frames Per Second (FPS)')
    axes[0].set_title('Speed Comparison - FPS', fontweight='bold', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    for i, (bar, value) in enumerate(zip(bars1, fps_values)):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{value:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # Inference Time Comparison
    bars2 = axes[1].bar(models, time_values, color=colors[:len(models)], alpha=0.7)
    axes[1].set_ylabel('Inference Time (milliseconds)')
    axes[1].set_title('Speed Comparison - Inference Time', fontweight='bold', fontsize=14)
    axes[1].grid(True, alpha=0.3)
    
    for i, (bar, value) in enumerate(zip(bars2, time_values)):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                    f'{value:.1f}ms', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# ===== 5. DETAILED ANALYSIS & CONCLUSIONS =====
print(f"\n🎯 DETAILED ANALYSIS & CONCLUSIONS")
print("=" * 60)

if len(comparison_results) >= 2:
    frcnn_fps = comparison_results.get('Faster R-CNN', {}).get('fps', 0)
    yolo_fps = comparison_results.get('YOLO', {}).get('fps', 0)
    
    print(f"🏃‍♂️ SPEED ANALYSIS:")
    if yolo_fps > frcnn_fps:
        speed_winner = "YOLO"
        speed_diff = ((yolo_fps - frcnn_fps) / frcnn_fps) * 100
        print(f"   ✅ Winner: YOLO ({speed_diff:.1f}% faster)")
        print(f"   📊 YOLO: {yolo_fps:.2f} FPS vs Faster R-CNN: {frcnn_fps:.2f} FPS")
    else:
        speed_winner = "Faster R-CNN"
        speed_diff = ((frcnn_fps - yolo_fps) / yolo_fps) * 100
        print(f"   ✅ Winner: Faster R-CNN ({speed_diff:.1f}% faster)")
        print(f"   📊 Faster R-CNN: {frcnn_fps:.2f} FPS vs YOLO: {yolo_fps:.2f} FPS")
    
    print(f"\n🎯 USE CASE RECOMMENDATIONS:")
    print(f"   🚗 Real-time applications (autonomous vehicles): YOLO")
    print(f"   🔬 High-accuracy research: Faster R-CNN")
    print(f"   📱 Mobile/edge devices: YOLO")
    print(f"   ☁️ Cloud-based batch processing: Faster R-CNN")

print(f"\n📝 GENERAL CHARACTERISTICS:")
print(f"🔸 Faster R-CNN:")
print(f"   ✅ Higher accuracy potential")
print(f"   ✅ Better for complex scenes")
print(f"   ❌ Slower inference")
print(f"   ❌ More memory intensive")

print(f"\n🔸 YOLO:")
print(f"   ✅ Real-time performance")
print(f"   ✅ Smaller model size")
print(f"   ✅ End-to-end training")
print(f"   ❌ May miss small objects")

print(f"\n🏁 FINAL CONCLUSION:")
print(f"For vehicle detection applications:")
print(f"   📊 Speed-critical systems: Choose YOLO")
print(f"   🎯 Accuracy-critical systems: Choose Faster R-CNN")
print(f"   ⚖️ Balanced requirements: Start with YOLO, fine-tune if needed")

print(f"\n✅ Comprehensive evaluation completed!")
clear_cuda_memory()

In [ ]:
# ===== GIẢI PHÁP TRÁNH TRÀN VRAM CUDA =====
import torch
import gc

# 1. Clear CUDA cache trước khi train
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"🧹 Cleared CUDA cache")
    print(f"💾 CUDA Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Clear Python garbage collector
gc.collect()
print("✅ Memory cleanup completed")

from ultralytics import YOLO

# 2. Sử dụng GPU với các tối ưu hóa VRAM
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"🎯 Using device: {device}")

model = YOLO("yolov8n.pt")  # Giữ yolov8n - model nhỏ nhất

# 3. Training với các kỹ thuật tiết kiệm VRAM
model.train(
    data="data.yaml", 
    epochs=50,
    # ===== CÁC THAM SỐ TIẾT KIỆM VRAM =====
    imgsz=416,         # Giảm từ 640 -> 416 (giảm ~50% VRAM)
    batch=4,           # Batch size nhỏ (mặc định thường là 16)
    amp=True,          # Mixed Precision Training (FP16) - giảm ~50% VRAM
    device=device,
    # ===== GRADIENT ACCUMULATION =====
    accumulate=4,      # Tích lũy gradient qua 4 batches = hiệu ứng batch=16
    # ===== CÁC THAM SỐ KHÁC =====
    workers=2,         # Giảm số worker processes
    patience=10,       # Early stopping nếu không cải thiện
    save_period=5,     # Lưu checkpoint mỗi 5 epochs
    cache=False,       # Không cache dataset vào RAM
    close_mosaic=10,   # Tắt mosaic augmentation ở 10 epoch cuối
)

In [ ]:
# ===== MONITOR CUDA MEMORY USAGE =====
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        print(f"💾 GPU Memory Usage:")
        print(f"   Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"   Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB") 
        print(f"   Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1024**3:.2f} GB")
        print(f"   Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        print("🚫 CUDA not available")

# Kiểm tra trước khi train
print_gpu_utilization()

In [ ]:
# ===== BACKUP PLAN NẾU VẪN BỊ TRÀN VRAM =====
# Nếu vẫn bị CUDA out of memory, chạy config này:

# from ultralytics import YOLO
# 
# # Cấu hình cực kỳ tiết kiệm VRAM
# model = YOLO("yolov8n.pt")
# model.train(
#     data="data.yaml",
#     epochs=50,
#     imgsz=320,        # Giảm xuống 320x320
#     batch=2,          # Batch size cực nhỏ
#     amp=True,         # Mixed precision
#     accumulate=8,     # Tích lũy nhiều gradient hơn
#     device=0,
#     workers=1,        # Chỉ 1 worker
#     cache=False,
#     mosaic=0.0,       # Tắt mosaic augmentation
#     mixup=0.0,        # Tắt mixup
#     copy_paste=0.0,   # Tắt copy-paste augmentation
#     degrees=0.0,      # Tắt rotation
#     translate=0.0,    # Tắt translation
#     scale=0.0,        # Tắt scaling
#     shear=0.0,        # Tắt shearing
#     flipud=0.0,       # Tắt vertical flip
#     fliplr=0.0,       # Tắt horizontal flip
#     hsv_h=0.0,        # Tắt hue augmentation
#     hsv_s=0.0,        # Tắt saturation augmentation
#     hsv_v=0.0         # Tắt value augmentation
# )

print("📝 Backup plan đã được chuẩn bị (uncomment nếu cần dùng)")